In [ ]:
#| default_exp plane

In [ ]:
#| export
from SolidPy.constants.constants import *

class Plane:
    def __init__(self, swModel, name, refPlane, offsetDistance, distanceUnits):
        self.model = swModel
        self.name = name
        self.refPlane = refPlane
        self.offsetDistance = offsetDistance
        self.distanceUnits = distanceUnits

    def create(self):
        # creates a plane after selecting the reference plane given in the function
        print("Creating plane")
        modelExt = self.model.Extension
        # modelExt = self.model.Extension
        featureMgr = self.model.FeatureManager
        distance = convertUnits(self.offsetDistance, self.distanceUnits)
        BoolStatus = modelExt.SelectByID2(
            self.refPlane, "PLANE", 0, 0, 0, False, 0, arg1, 0
        )
        # print(BoolStatus)
        print(distance < 0)
        if distance < 0:
            CreatePlane = featureMgr.InsertRefPlane(
                swConst.swRefPlaneReferenceConstraint_Distance | swConst.swRefPlaneReferenceConstraint_OptionFlip,
                distance,
                0,
                0,
                0,
                0,
            )
        else:
            CreatePlane = featureMgr.InsertRefPlane(
                swConst.swRefPlaneReferenceConstraint_Distance, distance, 0, 0, 0, 0
            )
        # print(CreatePlane)
        CreatePlane.Name = self.name
        self.model.ClearSelection2(True)
        return


    def edit(self):  # , swModel, refPlane, offsetDistance, distanceUnits):
        # edits a plane after selecting the reference plane given in the function
        try:
            modelExt = self.model.Extension
            # modelExt = self.model.Extension
            # featureMgr = self.model.FeatureManager
            swSelMgr = self.model.SelectionManager
            distance = convertUnits(self.offsetDistance, self.distanceUnits)
            BoolStatus = modelExt.SelectByID2(self.name, "PLANE", 0, 0, 0, False, 0, arg1, 0)
            # print(BoolStatus)
            Feature = swSelMgr.GetSelectedObject(1)
            swRefPlane = Feature.GetDefinition
            # swRefPlane.Distance = distance
            swRefPlane.AccessSelections(self.model, arg1)
            flip = swRefPlane.ReverseDirection  # ReversedReferenceDirection
            print(flip)
            if distance < 0:
                flip = True  # swConst.swRefPlaneReferenceConstraint_OptionFlip
            print(flip)
            swRefPlane.ReverseDirection = flip  # (swConst.swRefPlaneReference_First)
            swRefPlane.Distance = distance
            Feature.ModifyDefinition(swRefPlane, self.model, arg1)
            self.model.ClearSelection2(True)
            # print(BoolStatus)
            # CreatePlane = featureMgr.EditRefPlane(swConst.swRefPlaneReferenceConstraint_Distance, distance, 0, 0, 0, 0)
            # print(CreatePlane)
            # swModel.ClearSelection2(True)
        except:
            swRefPlane.ReleaseSelectionAccess
            print("Error editing plane: " + self.name)
        return

    def getProperties(self):
        try:
            # gets the properties of the plane
            properties = []  # "RefPlane"]
            swSelMgr = self.model.SelectionManager
            print("Getting properties of plane: " + self.name)
            BoolStatus = self.model.Extension.SelectByID2(
                self.name, "PLANE", 0, 0, 0, False, 0, arg1, 0
            )
            print(str(BoolStatus))
            if BoolStatus == False:
                print("Error selecting plane: " + self.name)
                return
            Feature = swSelMgr.GetSelectedObject6(1, 0)
            print(str(Feature))
            swRefPlane = Feature.GetDefinition
            swRefPlane.AccessSelections(self.model, arg1)
            # if swRefPlane.Selections != None:
            print(str(swRefPlane.Selections))
            try:
                properties.append(swRefPlane.Selections[0].Name)
            except:
                properties.append(swRefPlane.Selections.Name)

            properties.append(swRefPlane.Distance * 1000)
            # properties.append(swRefPlane.DistanceType)
            # properties.append(swRefPlane.DistanceUnits)
            # properties.append(swRefPlane.PlaneType)
            self.refPlane = swRefPlane.Selections[0].Name
            self.offsetDistance = swRefPlane.Distance * 1000
            # self.distanceUnits = properties[1]
            swRefPlane.ReleaseSelectionAccess()  # (self.model)
            return properties
        except:
            print("Error getting properties of plane: " + self.name)
            swRefPlane.ReleaseSelectionAccess()  # (self.model)
            return
        

